In [1]:
import torch

In [25]:
cos = torch.load('cos')
sin = torch.load('sin')

key_layer_before_causal = torch.load("key_layer_before_causal")
query_layer_before_causal = torch.load("query_layer_before_causal")
key_layer_after_causal = torch.load("key_layer_after_causal")
query_layer_after_causal = torch.load("query_layer_after_causal")

key_layer_before_flashinfer = torch.load("key_layer_before_flashinfer")
query_layer_before_flashinfer = torch.load("query_layer_before_flashinfer")


In [3]:
rotary_pos_emb_flashinfer = torch.cat((cos.transpose(1, 2), sin.transpose(1, 2)), dim=2)

In [26]:
import rotary_emb
def compute_rotary(x, cos, sin):
    rotary_dim = cos.shape[-1] * 2
    even_positions = list(range(0, rotary_dim, 2))
    odd_positions = list(range(1, rotary_dim, 2))

    k1 = x[..., even_positions]
    k2 = x[..., odd_positions]
    rotary_emb.apply_rotary(k1, k2, cos, sin, k1, k2, False)
    x[..., :rotary_dim] = torch.stack((k1, k2), dim=-1).flatten(start_dim=-2)

In [33]:
compute_rotary(key_layer_before_flashinfer, cos, sin)

In [39]:
key_layer_before_flashinfer.shape, key_layer_after_causal.shape
assert_close(key_layer_before_flashinfer.transpose(0, 1), key_layer_after_causal[0], 1e-4, 3e-5)

In [29]:
key_layer_before_causal.shape, cos.shape, sin.shape

(torch.Size([1, 2, 7, 128]), torch.Size([7, 1, 32]), torch.Size([7, 1, 32]))

In [30]:
rotary_dim = cos.shape[-1] * 2
even_positions = list(range(0, rotary_dim, 2))
odd_positions = list(range(1, rotary_dim, 2))
x = key_layer_before_causal

k1 = x[..., even_positions]
k2 = x[..., odd_positions]
rotary_emb.apply_rotary(k1, k2, cos, sin, k1, k2, False)
x[..., :rotary_dim] = torch.stack((k1, k2), dim=-1).flatten(start_dim=-2)

RuntimeError: The size of tensor a (2) must match the size of tensor b (7) at non-singleton dimension 1

In [32]:
k1.shape, k2.shape, cos.shape, sin.shape

(torch.Size([1, 2, 7, 32]),
 torch.Size([1, 2, 7, 32]),
 torch.Size([7, 1, 32]),
 torch.Size([7, 1, 32]))

In [27]:
head_dim = 8
rotary_dim = head_dim // 2
rotary_dim_half = rotary_dim // 2

cos1 = cos[6, 0, :rotary_dim_half].unsqueeze(0).unsqueeze(0)
sin1 = sin[6, 0, :rotary_dim_half].unsqueeze(0).unsqueeze(0)

k = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7], dtype = torch.float16, device=cos.device).unsqueeze(0).unsqueeze(0)
compute_rotary(k, cos1, sin1)
k
# k1 = k[..., :rotary_dim_half]
# k2 = k[..., rotary_dim_half : 2 * rotary_dim_half]




tensor([[[ 0.2793,  0.9600, -0.0879, -3.6035,  4.0000,  5.0000,  6.0000,
           7.0000]]], device='cuda:0', dtype=torch.float16)

In [16]:
even_positions + odd_positions
torch.stack((k1, k2), dim=-1).flatten(start_dim=-2)

tensor([[[ 0.2793,  0.9600, -0.0879, -3.6035]]], device='cuda:0',
       dtype=torch.float16)

In [11]:
k1.shape

torch.Size([1, 1, 2])

In [5]:
k1.shape, cos1.shape, sin1.shape, k2.shape

NameError: name 'k2' is not defined

In [8]:
[item for pair in zip(k1, k2) for item in pair]

[tensor([[ 0.2793, -0.0879]], device='cuda:0', dtype=torch.float16),
 tensor([[ 0.9600, -3.6035]], device='cuda:0', dtype=torch.float16)]

In [44]:
rotary_pos_emb = torch.cat((cos1.transpose(1, 2), sin1.transpose(1, 2)), dim=2).unsqueeze(0)
k_causal = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7], dtype = torch.float16, device=cos.device).unsqueeze(0).unsqueeze(0).unsqueeze(0)
apply_rotary_pos_emb(k_causal, rotary_pos_emb)

tensor([[[[[0., 1.],
           [2., 3.]]]]], device='cuda:0', dtype=torch.float16)


tensor([[[[ 0.2793,  0.9600, -0.0879, -3.6035,  4.0000,  5.0000,  6.0000,
            7.0000]]]], device='cuda:0', dtype=torch.float16)

In [38]:
rotary_pos_emb.shape, k_causal.shape

(torch.Size([1, 2, 2]), torch.Size([1, 1, 1, 8]))

In [42]:
def apply_rotary_pos_emb(x: torch.Tensor, rope_cache: torch.Tensor) -> torch.Tensor:
    # x: [b, np, sq, hn]
    b, np, sq, hn = x.size(0), x.size(1), x.size(2), x.size(3)
    rot_dim = rope_cache.shape[-2] * 2
    x, x_pass = x[..., :rot_dim], x[..., rot_dim:]
    # truncate to support variable sizes
    rope_cache = rope_cache[:, :sq]
    xshaped = x.reshape(b, np, sq, rot_dim // 2, 2)
    print(xshaped)
    rope_cache = rope_cache.view(-1, 1, sq, xshaped.size(3), 2)
    x_out2 = torch.stack(
        [
            xshaped[..., 0] * rope_cache[..., 0] - xshaped[..., 1] * rope_cache[..., 1],
            xshaped[..., 1] * rope_cache[..., 0] + xshaped[..., 0] * rope_cache[..., 1],
        ],
        -1,
    )
    x_out2 = x_out2.flatten(3)
    return torch.cat((x_out2, x_pass), dim=-1)

In [49]:
t1 = torch.tensor([0, 1, 2, 3, 4, 5]).reshape(3, 2).unsqueeze(0)
t1[:, 0], t1[..., 0], t1[:, 1], t1[..., 1]

(tensor([[0, 1]]), tensor([[0, 2, 4]]), tensor([[2, 3]]), tensor([[1, 3, 5]]))

In [50]:
t1

tensor([[[0, 1],
         [2, 3],
         [4, 5]]])

In [36]:
def assert_close(a, b, rtol, atol):
    # rtol, atol = {
    #     torch.float16: (1e-2, 5e-2),
    #     torch.bfloat16: (8e-3, 8e-3),
    # }[a.dtype]
    torch.testing.assert_close(a, b, rtol=rtol, atol=atol)